Name: Nikhil V

Roll No: CH.EN.U4AIE22038

COMPUTER SECURITY LAB

# Simplified Data Encryption Standard

In [2]:
def permute(input_bits, permutation):
    return [input_bits[i] for i in permutation]

In [3]:
def shift_left(bits, shifts):
    return bits[shifts:] + bits[:shifts]

In [6]:
def key_generation(key):

    # P10 permutation
    P10 = [2, 4, 1, 6, 3, 9, 0, 8, 7, 5]

    # P8 permutation
    P8 = [5, 2, 6, 3, 7, 4, 9, 8]

    key = permute(key, P10)

    # Split into left and right halves
    left, right = key[:5], key[5:]

    # Perform left shift by 1
    left = shift_left(left, 1)
    right = shift_left(right, 1)

    # Generate K1
    K1 = permute(left + right, P8)

    # Perform left shift by 2
    left = shift_left(left, 2)
    right = shift_left(right, 2)

    # Generate K2
    K2 = permute(left + right, P8)

    return K1, K2

In [7]:
def sbox(input_bits, sbox_table):
    row = (input_bits[0] << 1) | input_bits[3]
    col = (input_bits[1] << 1) | input_bits[2]
    return [int(b) for b in format(sbox_table[row][col], '02b')]

In [8]:
def fk(bits, subkey):

    # EP permutation
    EP = [3, 0, 1, 2, 1, 2, 3, 0]

    # S-boxes
    S0 = [[1, 0, 3, 2],
          [3, 2, 1, 0],
          [0, 2, 1, 3],
          [3, 1, 3, 2]]

    S1 = [[0, 1, 2, 3],
          [2, 0, 1, 3],
          [3, 0, 1, 0],
          [2, 1, 0, 3]]

    # P4 permutation
    P4 = [1, 3, 2, 0]

    left, right = bits[:4], bits[4:]

    expanded_permuted = permute(right, EP)

    xor_result = [b ^ k for b, k in zip(expanded_permuted, subkey)]

    left_xor, right_xor = xor_result[:4], xor_result[4:]

    s0_output = sbox(left_xor, S0)
    s1_output = sbox(right_xor, S1)

    sbox_output = permute(s0_output + s1_output, P4)

    return [l ^ s for l, s in zip(left, sbox_output)] + right

In [9]:
def encrypt(plaintext, key):

    # Initial permutation (IP)
    IP = [1, 5, 2, 0, 3, 7, 4, 6]

    # Inverse initial permutation (IP-1)
    IP_inv = [3, 0, 2, 4, 6, 1, 7, 5]

    # Generate subkeys
    K1, K2 = key_generation(key)

    bits = permute(plaintext, IP)

    # First round with K1
    bits = fk(bits, K1)

    # Swap halves
    bits = bits[4:] + bits[:4]

    # Second round with K2
    bits = fk(bits, K2)

    # Perform inverse initial permutation
    return permute(bits, IP_inv)

In [10]:
def decrypt(ciphertext, key):

    # Initial permutation (IP)
    IP = [1, 5, 2, 0, 3, 7, 4, 6]

    # Inverse initial permutation (IP-1)
    IP_inv = [3, 0, 2, 4, 6, 1, 7, 5]

    # Generate subkeys
    K1, K2 = key_generation(key)

    # Perform initial permutation
    bits = permute(ciphertext, IP)

    # First round with K2
    bits = fk(bits, K2)

    # Swap halves
    bits = bits[4:] + bits[:4]

    # Second round with K1
    bits = fk(bits, K1)

    # Perform inverse initial permutation
    return permute(bits, IP_inv)

In [11]:
if __name__ == "__main__":
    
    # 10-bit key and 8-bit plaintext
    key = [1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
    plaintext = [1, 0, 0, 1, 0, 1, 1, 1]

    print("Key:       ", key)
    print("Plaintext: ", plaintext)

    ciphertext = encrypt(plaintext, key)
    print("Ciphertext:", ciphertext)

    decrypted_text = decrypt(ciphertext, key)
    print("Decrypted: ", decrypted_text)

Key:        [1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
Plaintext:  [1, 0, 0, 1, 0, 1, 1, 1]
Ciphertext: [0, 0, 1, 1, 1, 0, 0, 0]
Decrypted:  [1, 0, 0, 1, 0, 1, 1, 1]
